In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from functools import *
import operator

dataset = pd.read_csv('../mnist/mnist_train.csv')
testset = pd.read_csv('../mnist/mnist_test.csv')

In [2]:
class Perceptron(object):
    def __init__(self,input_size, number=-1, lr=1, epochs=100):
        self.W = np.random.normal(0, 0.5, input_size+1)
        self.epochs = epochs
        self.lr = lr
        self.number = number
    
    def activation_fn(self, x):
        return (1 / (1 + np.exp(-x)))
 
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
    
    def guess_new(self, y):
        x = np.insert(y, 0, 1)
        return self.predict(x)
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [3]:
x_test = np.array(testset)[:, 1:] / 255
y_test = np.array(testset)[:, 0]

In [4]:
#Train perceptron by correspondent number
print(np.array(dataset).shape[0])
samples = 500
perseptrons = [Perceptron(input_size=784, lr=0.001, epochs=50,number=i) for i in range(10)]

for perceptron in perseptrons:
    x_train = np.array(dataset)[0:samples, :]
    
    #Selectors
    x_true_selector = perceptron.number == x_train[:, 0]    
    x_true_selector = pd.Series(x_true_selector)
    x_false_selector = perceptron.number != x_train[: ,0]
    x_false_selector = pd.Series(x_false_selector)
    
    # 50% True 50% False examples
    correspondent_x_train = x_train[x_true_selector]
    not_correspondent_x_train = x_train[x_false_selector]
    
    #Reduce length to match correspondent 
    not_correspondent_x_train = not_correspondent_x_train[0:correspondent_x_train.shape[0],:]
    
    x_train = np.concatenate((not_correspondent_x_train, correspondent_x_train))
    np.random.shuffle(x_train)
    
    #Final x_train, y_train values
    y_train = x_train[:, 0]
    y_train = [1 if(i == perceptron.number) else 0 for i in y_train]
    y_train = np.array(y_train)
    x_train = x_train[:, 1:] / 255
    
    #Fit Perceptron
    perceptron.fit(x_train, y_train)
    
    #Get Accuracy
    y_boolean_test = [1 if(i == perceptron.number) else 0 for i in y_test]
    y_boolean_test = np.array(y_boolean_test)
    
    total = y_test.shape[0]
    err_total = 0.0
    for test_case, test_answer in zip(x_test, y_boolean_test):
        prediction = perceptron.guess_new(test_case)
        err_total += abs(test_answer - prediction)
    print('Accuracy of Perceptron number ' + str(perceptron.number) + ' is: ' + str(100 - 100*err_total/total)[:4] + '%')

59999
Accuracy of Perceptron number 0 is: 40.8%
Accuracy of Perceptron number 1 is: 64.8%
Accuracy of Perceptron number 2 is: 43.3%
Accuracy of Perceptron number 3 is: 44.2%
Accuracy of Perceptron number 4 is: 42.6%
Accuracy of Perceptron number 5 is: 31.9%
Accuracy of Perceptron number 6 is: 75.5%
Accuracy of Perceptron number 7 is: 34.2%
Accuracy of Perceptron number 8 is: 36.8%
Accuracy of Perceptron number 9 is: 49.3%


In [5]:

acc_total = 0
total = y_test.shape[0]

for test_case, answer in zip(x_test, y_test):
    predictions = [perceptron.guess_new(test_case) for perceptron in perseptrons]
    if np.argmax(predictions) == answer:
        acc_total += 1
print('Accuracy of the network is: ' + str(100 * float(acc_total)/float(total))[:4] + '%')

Accuracy of the network is: 10.3%
